<a href="https://colab.research.google.com/github/Tom-Jung/Tom-Jung/blob/main/API_DART_F_%EC%97%91%EC%8A%A4%EC%97%A0%EC%97%98.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install requests pandas xml.etree.ElementTree

ERROR: Could not find a version that satisfies the requirement xml.etree.ElementTree (from versions: none)
ERROR: No matching distribution found for xml.etree.ElementTree


In [25]:
def download_audit_report(api_key, rcept_no):
    """감사보고서 다운로드 및 ZIP 해제"""
    url = f"https://opendart.fss.or.kr/api/document.xml?crtfc_key={api_key}&rcept_no={rcept_no}"
    response = requests.get(url)

    print(f"Response Status Code: {response.status_code}")
    print(f"Content-Type: {response.headers.get('Content-Type')}")

    if response.status_code == 200:
        try:
            with zipfile.ZipFile(io.BytesIO(response.content)) as z:
                file_names = z.namelist()
                print("ZIP 파일 내부 파일 목록:", file_names)

                with z.open(file_names[0]) as f:
                    xml_data = f.read().decode("utf-8")
                    print("XML 데이터 미리보기 (500자):", xml_data[:500])  # XML 일부 출력
                    return xml_data

        except zipfile.BadZipFile:
            print("Error: 응답이 ZIP 파일이 아닙니다.")
            return None
    else:
        print("Download Error:", response.status_code)
        return None


In [13]:
import requests
import zipfile
import io
import xml.etree.ElementTree as ET
from bs4 import BeautifulSoup  # HTML 파싱 추가

API_KEY = "85006d20472a255d5eae25ee265b5ed1b78d1386"
CORP_CODE = "01274727"  # 삼성전자 예제

def get_audit_report_list(api_key, corp_code, start_date, end_date):
    """감사보고서 목록 조회"""
    url = f"https://opendart.fss.or.kr/api/list.json?crtfc_key={api_key}&corp_code={corp_code}&bgn_de={start_date}&end_de={end_date}&kind=A001&sort=date"
    response = requests.get(url)
    data = response.json()

    if data["status"] == "000":
        return data["list"]
    else:
        print("API Error:", data["message"])
        return []

def download_audit_report(api_key, rcept_no):
    """감사보고서 다운로드 (ZIP/XML/HTML 자동 처리)"""
    url = f"https://opendart.fss.or.kr/api/document.xml?crtfc_key={api_key}&rcept_no={rcept_no}"
    response = requests.get(url)

    print(f"Response Status Code: {response.status_code}")
    print(f"Content-Type: {response.headers.get('Content-Type')}")

    if response.status_code == 200:
        # ZIP 파일 처리 (XML)
        if "application/x-msdownload" in response.headers.get('Content-Type', ''):
            try:
                with zipfile.ZipFile(io.BytesIO(response.content)) as z:
                    file_names = z.namelist()
                    print("ZIP 파일 내부 파일 목록:", file_names)
                    with z.open(file_names[0]) as f:
                        xml_data = f.read().decode("utf-8")
                        return "xml", xml_data, file_names[0]  # XML과 파일명 반환
            except zipfile.BadZipFile:
                print("Error: 응답이 ZIP 파일이 아닙니다.")
                return None, None, None

        # HTML 파일 처리
        elif "text/html" in response.headers.get('Content-Type', ''):
            html_data = response.text
            return "html", html_data, f"{rcept_no}.html"  # HTML과 파일명 반환

    print("Download Error:", response.status_code)
    return None, None, None

def save_audit_report(data_type, data, filename):
    """XML 또는 HTML 데이터를 파일로 저장"""
    if data and filename:
        with open(filename, "w", encoding="utf-8") as f:
            f.write(data)
        print(f"파일 저장 완료: {filename}")

def parse_audit_report(data_type, data):
    """데이터 파싱 (XML 또는 HTML 자동 구분)"""
    if data_type == "xml":
        try:
            # XML 자동 보정 (BeautifulSoup 사용)
            soup = BeautifulSoup(data, "xml")
            pretty_xml = soup.prettify()
            root = ET.fromstring(pretty_xml)

            for child in root.iter():
                print(child.tag, child.text)
        except ET.ParseError as e:
            print("XML Parsing Error:", e)

    elif data_type == "html":
        # HTML 파싱 (BeautifulSoup)
        soup = BeautifulSoup(data, "html.parser")
        print("HTML 제목:", soup.title.string if soup.title else "제목 없음")

        # 특정 내용 추출 예제
        texts = soup.get_text("\n", strip=True)  # 텍스트만 추출
        print("HTML 텍스트 미리보기:", texts[:500])

if __name__ == "__main__":
    start_date = "20240101"
    end_date = "20241231"

    reports = get_audit_report_list(API_KEY, CORP_CODE, start_date, end_date)

    if reports:
        rcept_no = reports[0]["rcept_no"]
        data_type, data, filename = download_audit_report(API_KEY, rcept_no)

        if data:
            save_audit_report(data_type, data, filename)  # XML 또는 HTML 파일 저장
            parse_audit_report(data_type, data)  # 데이터 파싱


스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
TE None
TE 
         -
        
TE None
TR 
        
TE 
         (2)유형자산(주석3,5,6,7)
        
TE None
TE 
         204,825,778,745
        
TE None
TE 
         55,576,812,360
        
TR 
        
TE 
         토지
        
TE 
         152,871,505,668
        
TE None
TE 
         47,158,324,741
        
TE None
TR 
        
TE 
         건물
        
TE 
         58,348,358,787
        
TE None
TE 
         13,037,174,818
        
TE None
TR 
        
TE 
         감가상각누계액
        
TE 
         (6,872,423,701)
        
TE None
TE 
         (4,983,832,549)
        
TE None
TR 
        
TE 
         구축물
        
TE 
         11,440,000
        
TE None
TE 
         11,440,000
        
TE None
TR 
        
TE 
         감가상각누계액
        
TE 
         (11,439,000)
        
TE None
TE 
         (11,439,000)
        
TE None
TR 
        
TE 
         차량운반구
        
TE 
         120,722,432
        
TE None
TE 
         88,162,887
        
TE None
TR 
        


In [27]:
import requests

# DART API 인증키
api_key = "85006d20472a255d5eae25ee265b5ed1b78d1386"

# 기업 코드 입력 (이미 알고 있는 값 사용)
corp_code = "01553767"  # 예시

# 감사보고서(사업보고서) 리스트 조회
list_url = f"https://opendart.fss.or.kr/api/list.json?crtfc_key={api_key}&corp_code={corp_code}&bgn_de=20230101&end_de=20231231&reprt_code=11011"

response = requests.get(list_url)
data = response.json()

if "list" in data:
    for item in data["list"]:
        if "rcept_no" in item:
            rcept_no = item["rcept_no"]
            print(f"📌 접수번호: {rcept_no}")
            break  # 가장 최신 보고서 1개만 사용
else:
    print("🚨 감사보고서 데이터 없음")


📌 접수번호: 20230403004324


#
:------------------------------------------------------------------------------엔클로니(normal,finance다포홈)------------------------------------

In [ ]:
from bs4 import BeautifulSoup
import pandas as pd

# XML 파일 로드 (파일 존재 여부 확인)
xml_path = "/content/20240412001149_00760.xml"
try:
    with open(xml_path, "r", encoding="utf-8") as file:
        xml_content = file.read()
except FileNotFoundError:
    print(f"❌ 파일을 찾을 수 없습니다: {xml_path}")
    exit()

# BeautifulSoup 파싱
soup = BeautifulSoup(xml_content, "xml")

# <COMPANY-NAME> 태그에서 회사명 추출
company_tag = soup.find("COMPANY-NAME")
company_name = company_tag.get_text(strip=True) if company_tag else "Unknown"
print(f"🏢 회사명: {company_name}")

# 특정 속성을 가진 <TABLE> 찾기 (NORMAL 또는 FINANCE)
target_tables = soup.find_all("TABLE", {"BORDER": "1", "AFIXTABLE": "N"})
filtered_tables = [
    table for table in target_tables if table.get("ACLASS") in {"NORMAL", "FINANCE"}
]

# 테이블 개수 확인
print(f"🔍 총 {len(target_tables)}개의 <TABLE> 태그를 찾았습니다.")
print(f"✅ 'NORMAL' 또는 'FINANCE' 필터링된 테이블 개수: {len(filtered_tables)}")

# 데이터 저장 리스트
data = []

# 각 <TABLE> 내부의 <TR> 및 <TD> 데이터 추출
for table in filtered_tables:
    rows = table.find_all("TR")  # <TR> 태그 찾기
    for row in rows:
        cols = row.find_all("TD")  # <TD> 태그 찾기
        row_data = [col.get_text(strip=True) for col in cols]  # 공백 제거 후 텍스트만 저장
        if row_data:
            # 각 행에 회사명 추가
            row_data.insert(0, company_name)
            data.append(row_data)

# Pandas DataFrame 생성 (첫 번째 컬럼을 회사명으로 추가)
df = pd.DataFrame(data)

# 출력 확인
print(df)

# CSV 파일로 저장
df.to_csv("output_filtered_NORMAL_FINANCE.csv", index=False, encoding="utf-8-sig")
print("✅ CSV 파일 저장 완료: output_filtered_NORMAL_FINANCE.csv")


#
:------------------------------------------------------------------------------엔클로니(
  finance포함
)------------------------------------

In [4]:
from bs4 import BeautifulSoup
import pandas as pd

# XML 파일 로드 (파일 존재 여부 확인)
xml_path = "/content/20240412001149_00760.xml"
try:
    with open(xml_path, "r", encoding="utf-8") as file:
        xml_content = file.read()
except FileNotFoundError:
    print(f"❌ 파일을 찾을 수 없습니다: {xml_path}")
    exit()

# BeautifulSoup 파싱
soup = BeautifulSoup(xml_content, "xml")

# <COMPANY-NAME> 태그에서 회사명 추출
company_tag = soup.find("COMPANY-NAME")
company_name = company_tag.get_text(strip=True) if company_tag else "Unknown"
print(f"🏢 회사명: {company_name}")

# 특정 속성을 가진 <TABLE> 찾기 (NORMAL 또는 FINANCE)
target_tables = soup.find_all("TABLE", {"BORDER": "1", "AFIXTABLE": "N"})
filtered_tables = [
    table for table in target_tables if table.get("ACLASS") in {"FINANCE"}
]

# 테이블 개수 확인
print(f"🔍 총 {len(target_tables)}개의 <TABLE> 태그를 찾았습니다.")
print(f"✅ 'NORMAL' 또는 'FINANCE' 필터링된 테이블 개수: {len(filtered_tables)}")

# 데이터 저장 리스트
data = []

# 각 <TABLE> 내부의 <TR> 및 <TD> 데이터 추출
for table in filtered_tables:
    rows = table.find_all("TR")  # <TR> 태그 찾기
    for row in rows:
        cols = row.find_all("TD")  # <TD> 태그 찾기
        row_data = [col.get_text(strip=True) for col in cols]  # 공백 제거 후 텍스트만 저장
        if row_data:
            # 각 행에 회사명 추가
            row_data.insert(0, company_name)
            data.append(row_data)

# Pandas DataFrame 생성 (첫 번째 컬럼을 회사명으로 추가)
df = pd.DataFrame(data)

# 출력 확인
print(df)

# CSV 파일로 저장
df.to_csv("output_filtered_NORMAL_FINANCE.csv", index=False, encoding="utf-8-sig")
print("✅ CSV 파일 저장 완료: output_filtered_NORMAL_FINANCE.csv")


🏢 회사명: (주)엔클로니
🔍 총 116개의 <TABLE> 태그를 찾았습니다.
✅ 'NORMAL' 또는 'FINANCE' 필터링된 테이블 개수: 0
Empty DataFrame
Columns: []
Index: []
✅ CSV 파일 저장 완료: output_filtered_NORMAL_FINANCE.csv


##------------------(주)대원에스엔씨

In [14]:
from bs4 import BeautifulSoup
import pandas as pd

# XML 파일 로드 (파일 존재 여부 확인)
xml_path = "/content/20240103000075_00760.xml"
try:
    with open(xml_path, "r", encoding="utf-8") as file:
        xml_content = file.read()
except FileNotFoundError:
    print(f"❌ 파일을 찾을 수 없습니다: {xml_path}")
    exit()

# BeautifulSoup 파싱
soup = BeautifulSoup(xml_content, "xml")

# <COMPANY-NAME> 태그에서 회사명 추출
company_tag = soup.find("COMPANY-NAME")
company_name = company_tag.get_text(strip=True) if company_tag else "Unknown"
print(f"🏢 회사명: {company_name}")

# 특정 속성을 가진 <TABLE> 찾기 (NORMAL 또는 FINANCE)
target_tables = soup.find_all("TABLE", {"BORDER": "1", "AFIXTABLE": "N"})
filtered_tables = [
    table for table in target_tables if table.get("ACLASS") in {"FINANCE"}
]

# 테이블 개수 확인
print(f"🔍 총 {len(target_tables)}개의 <TABLE> 태그를 찾았습니다.")
print(f"✅ 'NORMAL' 또는 'FINANCE' 필터링된 테이블 개수: {len(filtered_tables)}")

# 데이터 저장 리스트
data = []

# 각 <TABLE> 내부의 <TR> 및 <TD> 데이터 추출
for table in filtered_tables:
    rows = table.find_all("TR")  # <TR> 태그 찾기
    for row in rows:
        cols = row.find_all("TD")  # <TD> 태그 찾기
        row_data = [col.get_text(strip=True) for col in cols]  # 공백 제거 후 텍스트만 저장
        if row_data:
            # 각 행에 회사명 추가
            row_data.insert(0, company_name)
            data.append(row_data)

# Pandas DataFrame 생성 (첫 번째 컬럼을 회사명으로 추가)
df = pd.DataFrame(data)

# 출력 확인
print(df)

# CSV 파일로 저장
df.to_csv("output_filtered_NORMAL_FINANCE.csv", index=False, encoding="utf-8-sig")
print("✅ CSV 파일 저장 완료: output_filtered_NORMAL_FINANCE.csv")


🏢 회사명: (주)대원에스엔씨
🔍 총 30개의 <TABLE> 태그를 찾았습니다.
✅ 'NORMAL' 또는 'FINANCE' 필터링된 테이블 개수: 4
Empty DataFrame
Columns: []
Index: []
✅ CSV 파일 저장 완료: output_filtered_NORMAL_FINANCE.csv


In [15]:
from bs4 import BeautifulSoup
import pandas as pd

# XML 파일 로드 (파일 존재 여부 확인)
xml_path = "/content/20240103000075_00760.xml"
try:
    with open(xml_path, "r", encoding="utf-8") as file:
        xml_content = file.read()
except FileNotFoundError:
    print(f"❌ 파일을 찾을 수 없습니다: {xml_path}")
    exit()

# BeautifulSoup 파싱
soup = BeautifulSoup(xml_content, "xml")

# <COMPANY-NAME> 태그에서 회사명 추출
company_tag = soup.find("COMPANY-NAME")
company_name = company_tag.get_text(strip=True) if company_tag else "Unknown"
print(f"🏢 회사명: {company_name}")

# 특정 속성을 가진 <TABLE> 찾기 (NORMAL 또는 FINANCE)
target_tables = soup.find_all("TABLE", {"BORDER": "1", "AFIXTABLE": "N"})
filtered_tables = [
    table for table in target_tables if table.get("ACLASS") in {"FINANCE"}
]

# 테이블 개수 확인
print(f"🔍 총 {len(target_tables)}개의 <TABLE> 태그를 찾았습니다.")
print(f"✅ 'NORMAL' 또는 'FINANCE' 필터링된 테이블 개수: {len(filtered_tables)}")

# 데이터 저장 리스트
data = []

# 각 <TABLE> 내부의 <TR> 및 <TD> 데이터 추출
for table in filtered_tables:
    rows = table.find_all("TR")  # <TR> 태그 찾기
    for row in rows:
        cols = row.find_all("TD")  # <TD> 태그 찾기
        row_data = [col.get_text(strip=True) for col in cols]  # 공백 제거 후 텍스트만 저장
        if row_data:
            # 각 행에 회사명 추가
            row_data.insert(0, company_name)
            data.append(row_data)

# Pandas DataFrame 생성 (첫 번째 컬럼을 회사명으로 추가)
df = pd.DataFrame(data)

# 출력 확인
print(df)

# CSV 파일로 저장
df.to_csv("output_filtered_NORMAL_FINANCE.csv", index=False, encoding="utf-8-sig")
print("✅ CSV 파일 저장 완료: output_filtered_NORMAL_FINANCE.csv")


🏢 회사명: (주)대원에스엔씨
🔍 총 30개의 <TABLE> 태그를 찾았습니다.
✅ 'NORMAL' 또는 'FINANCE' 필터링된 테이블 개수: 4
Empty DataFrame
Columns: []
Index: []
✅ CSV 파일 저장 완료: output_filtered_NORMAL_FINANCE.csv


In [10]:
from bs4 import BeautifulSoup
import pandas as pd

# XML 파일 로드
xml_path = "/content/20240103000075_00760.xml"
try:
    with open(xml_path, "r", encoding="utf-8") as file:
        xml_content = file.read()
except FileNotFoundError:
    print(f"❌ 파일을 찾을 수 없습니다: {xml_path}")
    exit()

# XML 파싱
soup = BeautifulSoup(xml_content, "xml")

# 회사명 추출
company_tag = soup.find("COMPANY-NAME")
company_name = company_tag.get_text(strip=True) if company_tag else "Unknown"
print(f"🏢 회사명: {company_name}")

# 특정 테이블 필터링
target_tables = soup.find_all("TABLE", {"BORDER": "1", "AFIXTABLE": "N"})
filtered_tables = [table for table in target_tables if table.get("ACLASS") in {"FINANCE"}]

# 테이블 개수 확인
print(f"🔍 총 {len(target_tables)}개의 <TABLE> 태그를 찾았습니다.")
print(f"✅ 'FINANCE' 필터링된 테이블 개수: {len(filtered_tables)}")

# 데이터 저장 리스트
data = []

# 테이블 데이터 추출
for table in filtered_tables:
    rows = table.find_all("TR")
    print(f"📊 테이블 내 행 개수: {len(rows)}")  # ✅ 디버깅 추가

    if len(rows) == 0:
        continue  # 빈 테이블 스킵

    for row in rows:
        cols = row.find_all(["TD", "TH"])  # ✅ `TH` 포함
        if not cols:
            continue
        row_data = [col.get_text(strip=True) for col in cols]
        row_data.insert(0, company_name)  # ✅ 첫 번째 컬럼에 회사명 추가
        data.append(row_data)

# DataFrame 생성 및 CSV 저장
if data:
    df = pd.DataFrame(data)
    df.to_csv("output_filtered_NORMAL_FINANCE.csv", index=False, encoding="utf-8-sig")
    print("✅ CSV 파일 저장 완료: output_filtered_NORMAL_FINANCE.csv")
else:
    print("❌ 추출된 데이터가 없습니다. XML 구조를 다시 확인하세요.")


🏢 회사명: (주)대원에스엔씨
🔍 총 30개의 <TABLE> 태그를 찾았습니다.
✅ 'FINANCE' 필터링된 테이블 개수: 4
📊 테이블 내 행 개수: 52
📊 테이블 내 행 개수: 44
📊 테이블 내 행 개수: 11
📊 테이블 내 행 개수: 46
✅ CSV 파일 저장 완료: output_filtered_NORMAL_FINANCE.csv


#
:------------------------------------------------------------------------------큐로드------------------------------------



In [ ]:
from bs4 import BeautifulSoup
import pandas as pd

# XML 파일 로드 (파일 존재 여부 확인)
xml_path = "/content/20240403001942_00760.xml"
try:
    with open(xml_path, "r", encoding="utf-8") as file:
        xml_content = file.read()
except FileNotFoundError:
    print(f"❌ 파일을 찾을 수 없습니다: {xml_path}")
    exit()

# BeautifulSoup 파싱
soup = BeautifulSoup(xml_content, "xml")

# <COMPANY-NAME> 태그에서 회사명 추출
company_tag = soup.find("COMPANY-NAME")
company_name = company_tag.get_text(strip=True) if company_tag else "Unknown"
print(f"🏢 회사명: {company_name}")

# 특정 속성을 가진 <TABLE> 찾기 (NORMAL 또는 FINANCE)
target_tables = soup.find_all("TABLE", {"BORDER": "1", "AFIXTABLE": "N"})
filtered_tables = [
    table for table in target_tables if table.get("ACLASS") in {"NORMAL", "FINANCE"}
]

# 테이블 개수 확인
print(f"🔍 총 {len(target_tables)}개의 <TABLE> 태그를 찾았습니다.")
print(f"✅ 'NORMAL' 또는 'FINANCE' 필터링된 테이블 개수: {len(filtered_tables)}")

# 데이터 저장 리스트
data = []

# 각 <TABLE> 내부의 <TR> 및 <TD> 데이터 추출
for table in filtered_tables:
    rows = table.find_all("TR")  # <TR> 태그 찾기
    for row in rows:
        cols = row.find_all("TD")  # <TD> 태그 찾기
        row_data = [col.get_text(strip=True) for col in cols]  # 공백 제거 후 텍스트만 저장
        if row_data:
            # 각 행에 회사명 추가
            row_data.insert(0, company_name)
            data.append(row_data)

# Pandas DataFrame 생성 (첫 번째 컬럼을 회사명으로 추가)
df = pd.DataFrame(data)

# 출력 확인
print(df)

# CSV 파일로 저장
df.to_csv("output_filtered_NORMAL_FINANCE.csv", index=False, encoding="utf-8-sig")
print("✅ CSV 파일 저장 완료: output_filtered_NORMAL_FINANCE.csv")


#
:------------------------------------------------------------------------------우진기전------------------------------------

#
:------------------------------------------------------------------------------우진기전------------------------------------

In [17]:
from bs4 import BeautifulSoup
import pandas as pd

# XML 파일 로드 (파일 존재 여부 확인)
xml_path = "/content/20240411003596_00760.xml"
try:
    with open(xml_path, "r", encoding="utf-8") as file:
        xml_content = file.read()
except FileNotFoundError:
    print(f"❌ 파일을 찾을 수 없습니다: {xml_path}")
    exit()

# BeautifulSoup 파싱
soup = BeautifulSoup(xml_content, "xml")

# <COMPANY-NAME> 태그에서 회사명 추출
company_tag = soup.find("COMPANY-NAME")
company_name = company_tag.get_text(strip=True) if company_tag else "Unknown"
print(f"🏢 회사명: {company_name}")

# 특정 속성을 가진 <TABLE> 찾기 (NORMAL 또는 FINANCE)
target_tables = soup.find_all("TABLE", {"BORDER": "1", "AFIXTABLE": "N"})
filtered_tables = [
    table for table in target_tables if table.get("ACLASS") in {"NORMAL", "FINANCE"}
]

# 테이블 개수 확인
print(f"🔍 총 {len(target_tables)}개의 <TABLE> 태그를 찾았습니다.")
print(f"✅ 'NORMAL' 또는 'FINANCE' 필터링된 테이블 개수: {len(filtered_tables)}")

# 데이터 저장 리스트
data = []

# 각 <TABLE> 내부의 <TR> 및 <TD> 데이터 추출
for table in filtered_tables:
    rows = table.find_all("TR")  # <TR> 태그 찾기
    for row in rows:
        cols = row.find_all("TD")  # <TD> 태그 찾기
        row_data = [col.get_text(strip=True) for col in cols]  # 공백 제거 후 텍스트만 저장
        if row_data:
            # 각 행에 회사명 추가
            row_data.insert(0, company_name)
            data.append(row_data)

# Pandas DataFrame 생성 (첫 번째 컬럼을 회사명으로 추가)
df = pd.DataFrame(data)

# 출력 확인
print(df)

# CSV 파일로 저장
df.to_csv("output_filtered_NORMAL_FINANCE.csv", index=False, encoding="utf-8-sig")
print("✅ CSV 파일 저장 완료: output_filtered_NORMAL_FINANCE.csv")


🏢 회사명: (주)유진기전
🔍 총 31개의 <TABLE> 태그를 찾았습니다.
✅ 'NORMAL' 또는 'FINANCE' 필터링된 테이블 개수: 27
           0                                                  1          2  \
0    (주)유진기전  이 감사보고서는 감사보고서일 현재로 유효한 것입니다. 따라서 감사보고서일 후 이 보...       None   
1    (주)유진기전                                             건    물        40년   
2    (주)유진기전                                               기계장치         5년   
3    (주)유진기전                                              차량운반구          〃   
4    (주)유진기전                                                 비품          〃   
..       ...                                                ...        ...   
101  (주)유진기전                                              복리후생비    316,291   
102  (주)유진기전                                              감가상각비    413,775   
103  (주)유진기전                                              지급임차료      2,000   
104  (주)유진기전                                              세금과공과    129,237   
105  (주)유진기전                                               

In [16]:
from bs4 import BeautifulSoup
import pandas as pd

# XML 파일 로드 (파일 존재 여부 확인)
xml_path = "/content/20240411003596_00760.xml"
try:
    with open(xml_path, "r", encoding="utf-8") as file:
        xml_content = file.read()
except FileNotFoundError:
    print(f"❌ 파일을 찾을 수 없습니다: {xml_path}")
    exit()

# BeautifulSoup 파싱
soup = BeautifulSoup(xml_content, "xml")

# <COMPANY-NAME> 태그에서 회사명 추출
company_tag = soup.find("COMPANY-NAME")
company_name = company_tag.get_text(strip=True) if company_tag else "Unknown"
print(f"🏢 회사명: {company_name}")

# 특정 속성을 가진 <TABLE> 찾기 (NORMAL 또는 FINANCE)
target_tables = soup.find_all("TABLE", {"BORDER": "1", "AFIXTABLE": "N"})
filtered_tables = [
    table for table in target_tables if table.get("ACLASS") in {"NORMAL", "FINANCE"}
]

# 테이블 개수 확인
print(f"🔍 총 {len(target_tables)}개의 <TABLE> 태그를 찾았습니다.")
print(f"✅ 'NORMAL' 또는 'FINANCE' 필터링된 테이블 개수: {len(filtered_tables)}")

# 데이터 저장 리스트
data = []

# 각 <TABLE> 내부의 <TR> 및 <TD> 데이터 추출
for table in filtered_tables:
    rows = table.find_all("TR")  # <TR> 태그 찾기
    for row in rows:
        cols = row.find_all("TD")  # <TD> 태그 찾기
        row_data = [col.get_text(strip=True) for col in cols]  # 공백 제거 후 텍스트만 저장
        if row_data:
            # 각 행에 회사명 추가
            row_data.insert(0, company_name)
            data.append(row_data)

# Pandas DataFrame 생성 (첫 번째 컬럼을 회사명으로 추가)
df = pd.DataFrame(data)

# 출력 확인
print(df)

# CSV 파일로 저장
df.to_csv("output_filtered_NORMAL_FINANCE.csv", index=False, encoding="utf-8-sig")
print("✅ CSV 파일 저장 완료: output_filtered_NORMAL_FINANCE.csv")


🏢 회사명: (주)유진기전
🔍 총 31개의 <TABLE> 태그를 찾았습니다.
✅ 'NORMAL' 또는 'FINANCE' 필터링된 테이블 개수: 4
Empty DataFrame
Columns: []
Index: []
✅ CSV 파일 저장 완료: output_filtered_NORMAL_FINANCE.csv


#
:------------------------------------------------------------------------------슈랜드------------------------------------


In [ ]:
import re
import pandas as pd

# ✅ 텍스트 XML 파일 로드
file_path = "/content/20240409002195_00760.xml"  # 다운로드한 XML 텍스트 파일
try:
    with open(file_path, "r", encoding="utf-8") as file:
        xml_text = file.read()
except FileNotFoundError:
    print(f"❌ 파일을 찾을 수 없습니다: {file_path}")
    exit()

# ✅ 회사명 추출
company_name_match = re.search(r"COMPANY-NAME:\s*(.+)", xml_text)
company_name = company_name_match.group(1) if company_name_match else "Unknown"
print(f"🏢 회사명: {company_name}")

# ✅ 재무제표 섹션 정의
sections = ["재무상태표", "손익계산서", "현금흐름표"]
data = []
current_section = None

# ✅ 줄 단위로 데이터 읽기
for line in xml_text.split("\n"):
    line = line.strip()  # 공백 제거

    # ✅ 현재 섹션(재무상태표, 손익계산서, 현금흐름표) 체크
    if line in sections:
        current_section = line
        continue  # 섹션명 자체는 저장하지 않음

    # ✅ 숫자가 포함된 데이터 추출 (예: "매출액: 500000")
    match = re.match(r"(.+):\s*([\d-]+)", line)
    if match and current_section:
        item, value = match.groups()
        data.append([company_name, current_section, item, value])

# ✅ Pandas DataFrame 생성
df = pd.DataFrame(data, columns=["회사명", "재무제표 유형", "항목명", "금액"])

# ✅ CSV 파일 저장
df.to_csv("financial_statements.csv", index=False, encoding="utf-8-sig")
print("✅ CSV 파일 저장 완료: financial_statements.csv")

# ✅ 출력 확인
print(df)


🏢 회사명: Unknown
✅ CSV 파일 저장 완료: financial_statements.csv
Empty DataFrame
Columns: [회사명, 재무제표 유형, 항목명, 금액]
Index: []


In [ ]:
from bs4 import BeautifulSoup
import pandas as pd

# ✅ XML 파일 로드
xml_path = "/content/20240321001559_00760.xml"

try:
    with open(xml_path, "r", encoding="utf-8") as file:
        xml_content = file.read()
except FileNotFoundError:
    print(f"❌ 파일을 찾을 수 없습니다: {xml_path}")
    exit()

# ✅ BeautifulSoup 파싱
soup = BeautifulSoup(xml_content, "xml")

# ✅ 회사명 추출
company_tag = soup.find("COMPANY-NAME")
company_name = company_tag.get_text(strip=True) if company_tag else "Unknown"
print(f"🏢 회사명: {company_name}")

# ✅ 주요 재무 데이터 추출
financial_data = {}
summary_section = soup.find("SUMMARY")

if summary_section:
    for extraction in summary_section.find_all("EXTRACTION"):
        key = extraction.get("ACODE")  # ACODE 속성을 키로 사용
        value = extraction.get_text(strip=True)
        financial_data[key] = value

# ✅ 관심 있는 항목 필터링 (재무제표 관련)
financial_items = {
    "TOT_ASSETS": "총자산",
    "TOT_DEBTS": "총부채",
    "TOT_SALES": "총매출",
    "TOT_EMPL": "총직원수"
}

# ✅ 정리된 데이터 저장
data = [[company_name, financial_items.get(key, key), financial_data[key]] for key in financial_items if key in financial_data]

# ✅ Pandas DataFrame 생성
df = pd.DataFrame(data, columns=["회사명", "항목", "금액"])

# ✅ CSV 저장
df.to_csv("financial_summary.csv", index=False, encoding="utf-8-sig")
print("✅ CSV 파일 저장 완료: financial_summary.csv")

# ✅ 출력 확인
print(df)


🏢 회사명: (주)우진기전
✅ CSV 파일 저장 완료: financial_summary.csv
       회사명    항목      금액
0  (주)우진기전   총자산  164747
1  (주)우진기전   총부채   69363
2  (주)우진기전   총매출  252747
3  (주)우진기전  총직원수     143


In [ ]:
import xml.etree.ElementTree as ET

# XML 파일 읽기
tree = ET.parse("/content/20240321001559_00760.xml")
root = tree.getroot()  # 루트 요소 가져오기

# 루트 태그 출력
print("Root tag:", root.tag)

# XML 데이터 탐색
for employee in root.findall("자산"):
    emp_id = employee.get("id")  # 속성 가져오기
    name = employee.find("name").text
    position = employee.find("position").text
    salary = employee.find("salary").text
    print(f"Employee {emp_id}: {name}, {position}, Salary: {salary}")


ParseError: mismatched tag: line 2343, column 35 (<string>)

##--------------------pdf 다운로드-----------------------------

In [2]:
!pip install requests pandas xmltodict


In [3]:
import requests
import pandas as pd
import xml.etree.ElementTree as ET

# DART API 키
API_KEY = "85006d20472a255d5eae25ee265b5ed1b78d1386"

# 비상장 기업 (우진기전) 코드 (정확한 코드 사용!)
COMPANY_CODE = "00492326"  # 8자리 코드 확인 필수!

# API 요청 URL (pblntf_ty 삭제, 최신 데이터 범위 설정)
URL = f"https://opendart.fss.or.kr/api/list.xml?crtfc_key={API_KEY}&corp_code={COMPANY_CODE}&bgn_de=20240101&end_de=20241231&page_no=1&page_count=100"

# API 요청
response = requests.get(URL)
if response.status_code == 200:
    root = ET.fromstring(response.content)

    # 감사보고서 필터링
    data = []
    for report in root.findall("list"):
        report_nm = report.find("report_nm").text  # 보고서명

        if "감사보고서" in report_nm:  # 감사보고서만 필터링
            rcp_no = report.find("rcept_no").text  # 접수번호
            flr_nm = report.find("flr_nm").text  # 제출자명
            rcp_dt = report.find("rcept_dt").text  # 접수일자
            data.append([rcp_no, report_nm, flr_nm, rcp_dt])

    if data:
        df = pd.DataFrame(data, columns=["접수번호", "보고서명", "제출자명", "접수일자"])
        print(df)
    else:
        print("해당 기간 내 감사보고서 없음")
else:
    print("API 요청 실패:", response.status_code)


             접수번호             보고서명    제출자명      접수일자
0  20240321001559  감사보고서 (2023.12)  삼일회계법인  20240321


In [4]:
import requests
import pandas as pd
import xml.etree.ElementTree as ET
import os
import re
from bs4 import BeautifulSoup
import webbrowser

# DART API 키
API_KEY = "85006d20472a255d5eae25ee265b5ed1b78d1386"

# 비상장 기업 (우진기전) 코드 (정확한 코드 입력)
COMPANY_CODE = "00492326"  # 기업 코드 (8자리 확인 필요!)

# API 요청 URL (pblntf_ty 삭제, 최신 데이터 범위 설정)
URL = f"https://opendart.fss.or.kr/api/list.xml?crtfc_key={API_KEY}&corp_code={COMPANY_CODE}&bgn_de=20240101&end_de=20241231&page_no=1&page_count=100"

# 요청 헤더 (DART 서버가 참조 URL을 확인하기 때문)
HEADERS = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/110.0.0.0 Safari/537.36"
}

# API 요청
response = requests.get(URL)
if response.status_code == 200:
    root = ET.fromstring(response.content)

    # 감사보고서 필터링
    data = []
    for report in root.findall("list"):
        report_nm = report.find("report_nm").text  # 보고서명

        if "감사보고서" in report_nm:  # 감사보고서만 필터링
            rcp_no = report.find("rcept_no").text  # 접수번호
            flr_nm = report.find("flr_nm").text  # 제출자명
            rcp_dt = report.find("rcept_dt").text  # 접수일자
            report_url = f"https://dart.fss.or.kr/dsaf001/main.do?rcpNo={rcp_no}"  # 보고서 원문 URL

            data.append([rcp_no, report_nm, flr_nm, rcp_dt, report_url])

    if data:
        df = pd.DataFrame(data, columns=["접수번호", "보고서명", "제출자명", "접수일자", "보고서 URL"])
        print(df)

        # 📌 첫 번째 감사보고서 다운로드 (예제)
        first_rcp_no = df.iloc[0]["접수번호"]
        dart_url = f"https://dart.fss.or.kr/dsaf001/main.do?rcpNo={first_rcp_no}"  # 원문 페이지

        # 공시 본문 페이지 크롤링하여 PDF 다운로드 URL 찾기
        print(f"공시 본문 페이지 분석 중: {dart_url}")
        response = requests.get(dart_url, headers=HEADERS)
        if response.status_code == 200:
            soup = BeautifulSoup(response.text, "html.parser")
            pdf_link_tag = soup.find("a", text=re.compile("감사보고서"))  # 감사보고서 PDF 링크 찾기

            if pdf_link_tag:
                pdf_link = pdf_link_tag["href"]
                full_pdf_url = f"https://dart.fss.or.kr{pdf_link}"
                save_path = os.path.abspath(f"감사보고서_{first_rcp_no}.pdf")

                print(f"다운로드 중: {full_pdf_url}")

                # PDF 다운로드
                pdf_response = requests.get(full_pdf_url, headers={"Referer": dart_url})  # Referer 추가
                if pdf_response.status_code == 200:
                    with open(save_path, "wb") as f:
                        f.write(pdf_response.content)
                    print(f"감사보고서 저장 완료: {save_path}")

                    # 📌 Chrome에서 PDF 열기
                    webbrowser.open(save_path)  # 기본 브라우저로 열기
                else:
                    print("감사보고서 다운로드 실패")
            else:
                print("감사보고서 PDF 링크를 찾을 수 없음")
        else:
            print("공시 본문 페이지 요청 실패")

    else:
        print("해당 기간 내 감사보고서 없음")

else:
    print("API 요청 실패:", response.status_code)


             접수번호             보고서명    제출자명      접수일자  \
0  20240321001559  감사보고서 (2023.12)  삼일회계법인  20240321   

                                             보고서 URL  
0  https://dart.fss.or.kr/dsaf001/main.do?rcpNo=2...  
공시 본문 페이지 분석 중: https://dart.fss.or.kr/dsaf001/main.do?rcpNo=20240321001559
감사보고서 PDF 링크를 찾을 수 없음


<ipython-input-4-2f386ca3e8d9>:54: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  pdf_link_tag = soup.find("a", text=re.compile("감사보고서"))  # 감사보고서 PDF 링크 찾기


In [5]:
import requests
import pandas as pd
import xml.etree.ElementTree as ET
import os
from bs4 import BeautifulSoup
import webbrowser

# DART API 키
API_KEY = "85006d20472a255d5eae25ee265b5ed1b78d1386"

# 비상장 기업 (우진기전) 코드 (정확한 코드 입력)
COMPANY_CODE = "00492326"  # 기업 코드 (8자리 확인 필요!)

# API 요청 URL
URL = f"https://opendart.fss.or.kr/api/list.xml?crtfc_key={API_KEY}&corp_code={COMPANY_CODE}&bgn_de=20240101&end_de=20241231&page_no=1&page_count=100"

# 요청 헤더 (DART 서버에서 차단되지 않도록 설정)
HEADERS = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/110.0.0.0 Safari/537.36"
}

# API 요청
response = requests.get(URL)
if response.status_code == 200:
    root = ET.fromstring(response.content)

    # 감사보고서 필터링
    data = []
    for report in root.findall("list"):
        report_nm = report.find("report_nm").text  # 보고서명

        if "감사보고서" in report_nm:  # 감사보고서만 필터링
            rcp_no = report.find("rcept_no").text  # 접수번호
            flr_nm = report.find("flr_nm").text  # 제출자명
            rcp_dt = report.find("rcept_dt").text  # 접수일자
            report_url = f"https://dart.fss.or.kr/dsaf001/main.do?rcpNo={rcp_no}"  # 보고서 원문 URL

            data.append([rcp_no, report_nm, flr_nm, rcp_dt, report_url])

    if data:
        df = pd.DataFrame(data, columns=["접수번호", "보고서명", "제출자명", "접수일자", "보고서 URL"])
        print(df)

        # 📌 첫 번째 감사보고서 HTML 다운로드 (예제)
        first_rcp_no = df.iloc[0]["접수번호"]
        dart_url = f"https://dart.fss.or.kr/dsaf001/main.do?rcpNo={first_rcp_no}"  # 원문 페이지

        print(f"공시 본문 페이지 분석 중: {dart_url}")
        response = requests.get(dart_url, headers=HEADERS)
        if response.status_code == 200:
            html_content = response.text

            # HTML 파일 저장
            save_path = os.path.abspath(f"감사보고서_{first_rcp_no}.html")
            with open(save_path, "w", encoding="utf-8") as f:
                f.write(html_content)
            print(f"감사보고서 HTML 저장 완료: {save_path}")

            # 📌 Chrome에서 HTML 열기
            webbrowser.open(save_path)  # 기본 브라우저로 열기
        else:
            print("공시 본문 페이지 요청 실패")

    else:
        print("해당 기간 내 감사보고서 없음")

else:
    print("API 요청 실패:", response.status_code)


             접수번호             보고서명    제출자명      접수일자  \
0  20240321001559  감사보고서 (2023.12)  삼일회계법인  20240321   

                                             보고서 URL  
0  https://dart.fss.or.kr/dsaf001/main.do?rcpNo=2...  
공시 본문 페이지 분석 중: https://dart.fss.or.kr/dsaf001/main.do?rcpNo=20240321001559
감사보고서 HTML 저장 완료: /content/감사보고서_20240321001559.html


###------------------pdf에서 변환

In [2]:
!pip install pdfplumber

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.5/42.5 kB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.5/59.5 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 55.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 58.3 MB/s eta 0:00:00


In [6]:
import pdfplumber
import pandas as pd

pdf_path = "/content/엔클로니_감사보고서.pdf"

data = []
with pdfplumber.open(pdf_path) as pdf:
    for page in pdf.pages:
        tables = page.extract_table()
        if tables:
            data.extend(tables)

# DataFrame으로 변환 후 저장
df = pd.DataFrame(data)
df.to_csv("output_엔클로니.csv", index=False)
df.to_excel("output_엔클로니.xlsx", index=False)

In [8]:
import pdfplumber
import pandas as pd

pdf_path = "/content/대원에스엔씨_감사보고서.pdf"

data = []
with pdfplumber.open(pdf_path) as pdf:
    for page in pdf.pages:
        tables = page.extract_table()
        if tables:
            data.extend(tables)

# DataFrame으로 변환 후 저장
df = pd.DataFrame(data)
df.to_csv("output_대원에스엔씨.csv", index=False)
df.to_excel("output_대원에스엔씨.xlsx", index=False)

In [10]:
import pdfplumber
import pandas as pd

pdf_path = "/content/우진기전_사보고서.pdf"

data = []
with pdfplumber.open(pdf_path) as pdf:
    for page in pdf.pages:
        tables = page.extract_table()
        if tables:
            data.extend(tables)

# DataFrame으로 변환 후 저장
df = pd.DataFrame(data)
df.to_csv("output_우진기전.csv", index=False)
df.to_excel("output_우진기전.xlsx", index=False)

In [11]:
import pdfplumber
import re
import pandas as pd

# 📌 PDF 파일 경로
pdf_path = "/content/우진기전_사보고서.pdf"

# 📌 추출된 데이터 저장 리스트
lease_values = []

# 📌 PDF에서 '리스' 뒤의 숫자 추출
with pdfplumber.open(pdf_path) as pdf:
    for page_num, page in enumerate(pdf.pages, start=1):
        text = page.extract_text()  # 페이지의 텍스트 추출
        if text:
            # ✅ 정규식 사용: '리스' 뒤에 오는 숫자 찾기 (콤마 포함 가능)
            matches = re.findall(r'리스\s*([\d,]+)', text)
            lease_values.extend(matches)

# 📌 데이터프레임 변환 후 저장
if lease_values:
    df = pd.DataFrame(lease_values, columns=["리스 금액"])
    df.to_csv("lease_values.csv", index=False, encoding="utf-8-sig")
    df.to_excel("lease_values.xlsx", index=False)

    print("✅ '리스' 뒤의 숫자 추출 완료: lease_values.csv, lease_values.xlsx")
else:
    print("❌ '리스' 뒤의 숫자를 찾을 수 없습니다.")


❌ '리스' 뒤의 숫자를 찾을 수 없습니다.


In [9]:
import pdfplumber
import pandas as pd

pdf_path = "/content/엔클로니_감사보고서.pdf"

data = []
with pdfplumber.open(pdf_path) as pdf:
    for page in pdf.pages:
        tables = page.extract_tables()  # 여러 개의 테이블을 처리하도록 변경
        if tables:
            for table in tables:
                if table:  # 빈 테이블이 아닐 경우만 추가
                    data.extend(table)

# DataFrame으로 변환 (컬럼명을 설정할 수도 있음)
df = pd.DataFrame(data)

# 컬럼명이 없을 경우, 첫 번째 행을 헤더로 설정 가능
df.columns = df.iloc[0]  # 첫 번째 행을 컬럼명으로 사용
df = df[1:]  # 첫 번째 행 제거

# 파일 저장
df.to_csv("output_엔클로니_f.csv", index=False, encoding="utf-8-sig")
df.to_excel("output_엔클로니_f.xlsx", index=False)

print("변환 완료: output_엔클로니.csv, output_엔클로니.xlsx")


변환 완료: output_엔클로니.csv, output_엔클로니.xlsx


In [17]:
import pdfplumber
import pandas as pd

pdf_path = "/content/엔클로니_감사보고서.pdf"

data = []
with pdfplumber.open(pdf_path) as pdf:
    for page in pdf.pages:
        tables = page.extract_tables()  # 여러 개의 테이블을 리스트로 반환
        if tables:  # 테이블이 존재하는 경우에만 처리
            for table in tables:
                if table:  # 빈 테이블 방지
                    data.extend(table)  # 리스트 내부 리스트를 평탄화하여 추가

# DataFrame으로 변환
df = pd.DataFrame(data)

# 컬럼명이 없는 경우, 첫 번째 행을 컬럼명으로 설정 가능
df.columns = df.iloc[0]  # 첫 번째 행을 컬럼명으로 설정
df = df[1:].reset_index(drop=True)  # 첫 번째 행 제거 후 인덱스 리셋

# 파일 저장
df.to_csv("output_엔클로니ff.csv", index=False, encoding="utf-8-sig")
df.to_excel("output_엔클로니ff.xlsx", index=False)

print("✅ 테이블 데이터만 추출 완료: output_엔클로니.csv, output_엔클로니.xlsx")


✅ 테이블 데이터만 추출 완료: output_엔클로니.csv, output_엔클로니.xlsx


In [14]:
import pdfplumber
import pandas as pd

pdf_path = "/content/엔클로니_감사보고서.pdf"

data = []
with pdfplumber.open(pdf_path) as pdf:
    for page in pdf.pages:
        tables = page.extract_tables()  # 페이지에서 모든 테이블 추출
        if tables:
            data.extend(tables)  # 리스트 평탄화하여 추가

# 상위 5개 테이블만 선택
data = data[:8]

# 각 테이블을 DataFrame으로 변환 후 하나로 합치기
dfs = [pd.DataFrame(table) for table in data if table]  # 빈 테이블 제외
df_combined = pd.concat(dfs, ignore_index=True)  # 하나의 DataFrame으로 합치기

# 파일 저장
df_combined.to_csv("output_엔클로니_combined.csv", index=False, encoding="utf-8-sig")
df_combined.to_excel("output_엔클로니_combined.xlsx", index=False)

print("✅ 상위 5개 테이블을 하나의 파일로 저장 완료: output_엔클로니_combined.csv, output_엔클로니_combined.xlsx")


✅ 상위 5개 테이블을 하나의 파일로 저장 완료: output_엔클로니_combined.csv, output_엔클로니_combined.xlsx


In [21]:
import pdfplumber
import pandas as pd

pdf_path = "/content/엔클로니_감사보고서.pdf"

data = []
with pdfplumber.open(pdf_path) as pdf:
    for page_num, page in enumerate(pdf.pages, start=1):
        tables = page.extract_tables()  # 페이지에서 테이블 추출
        if tables:
            for table in tables:
                if table:
                    df = pd.DataFrame(table)

                    # 컬럼명이 없거나 중복되었을 경우 기본 인덱스 부여
                    if df.shape[0] > 0:  # 테이블이 비어있지 않은 경우
                        if df.iloc[0].duplicated().sum() > 0 or df.iloc[0].isna().sum() > 0:
                            df.columns = [f"Column_{i}" for i in range(len(df.columns))]  # 컬럼명 자동 설정
                        else:
                            df.columns = df.iloc[0]  # 첫 번째 행을 컬럼명으로 설정
                            df = df[1:].reset_index(drop=True)  # 첫 번째 행 제거 후 인덱스 재정렬

                        data.append(df)

# 📌 상위 8개 테이블만 선택
dfs = data[:8]

# 📌 특정 항목 필터링 (자산총계, 부채총계, 자본총계, 매출액)
keywords = ["자산총계", "부채총계", "자본총계", "매출액"]
filtered_data = []

for df in dfs:
    df.dropna(how="all", axis=0, inplace=True)  # 빈 행 제거
    df.dropna(how="all", axis=1, inplace=True)  # 빈 열 제거

    # 특정 키워드가 포함된 행만 필터링
    if df.shape[0] > 0 and df.shape[1] > 0:  # 데이터가 존재할 경우만 진행
        df_filtered = df[df.iloc[:, 0].astype(str).str.contains("|".join(keywords), na=False)]

        if not df_filtered.empty:  # 빈 데이터프레임이 아닐 경우만 추가
            filtered_data.append(df_filtered)

# 📌 필터링된 테이블을 하나의 DataFrame으로 합치기
if filtered_data:
    df_combined = pd.concat(filtered_data, ignore_index=True)

    # 📌 파일 저장
    df_combined.to_csv("output_엔클로니_filtered.csv", index=False, encoding="utf-8-sig")
    df_combined.to_excel("output_엔클로니_filtered.xlsx", index=False)

    print("✅ 상위 8개 테이블에서 특정 항목만 추출 완료: output_엔클로니_filtered.csv, output_엔클로니_filtered.xlsx")
else:
    print("⚠️ 필터링된 데이터가 없습니다.")


✅ 상위 8개 테이블에서 특정 항목만 추출 완료: output_엔클로니_filtered.csv, output_엔클로니_filtered.xlsx


In [24]:
import pdfplumber
import pandas as pd
import re

pdf_path = "/content/엔클로니_감사보고서.pdf"

data = []
with pdfplumber.open(pdf_path) as pdf:
    for page_num, page in enumerate(pdf.pages, start=1):
        tables = page.extract_tables()  # 페이지에서 테이블 추출
        if tables:
            for table in tables:
                if table:
                    df = pd.DataFrame(table)
                    df.dropna(how="all", axis=0, inplace=True)  # 빈 행 제거
                    df.dropna(how="all", axis=1, inplace=True)  # 빈 열 제거

                    # 📌 모든 행을 하나의 컬럼으로 변환 (항목 + 금액들)
                    df_merged = df.astype(str).apply(lambda row: " ".join(row.dropna()), axis=1).to_frame(name="RawData")

                    data.append(df_merged)

# 📌 상위 8개 테이블만 선택
dfs = data[:8]

# 📌 특정 항목 필터링 (자산총계, 부채총계, 자본총계, 매출액)
keywords = ["자산총계", "부채총계", "자본총계", "매출액"]
filtered_data = []

for df in dfs:
    df_filtered = df[df["RawData"].str.contains("|".join(keywords), na=False)]

    if not df_filtered.empty:
        filtered_data.append(df_filtered)

# 📌 필터링된 데이터를 하나의 DataFrame으로 합치기
if filtered_data:
    df_combined = pd.concat(filtered_data, ignore_index=True)

    # 📌 항목과 금액들을 분리하는 함수
    def split_text_amounts(row):
        parts = row.split()  # 공백 기준으로 나누기
        text_part = parts[0]  # 첫 번째는 항목명 (예: 자산총계)
        amount_parts = [p.replace(",", "") for p in parts[1:] if re.match(r"^\d{1,3}(,\d{3})*$", p)]  # 금액만 추출
        return [text_part] + amount_parts  # 리스트 형태로 반환

    # 📌 데이터 정리 (항목 + 금액 여러 개 분리)
    split_data = df_combined["RawData"].apply(split_text_amounts)
    max_columns = max(split_data.apply(len))  # 가장 많은 금액 개수를 기준으로 컬럼 확장

    # 📌 확장된 데이터를 DataFrame으로 변환
    df_final = pd.DataFrame(split_data.tolist(), columns=["항목"] + [f"금액_{i+1}" for i in range(max_columns - 1)])

    # 📌 파일 저장
    df_final.to_csv("output_엔클로니_filtered.csv", index=False, encoding="utf-8-sig")
    df_final.to_excel("output_엔클로니_filtered.xlsx", index=False)

    print("✅ 상위 8개 테이블에서 특정 항목 및 모든 금액을 개별 셀로 저장 완료: output_엔클로니_filtered.csv, output_엔클로니_filtered.xlsx")
else:
    print("⚠️ 필터링된 데이터가 없습니다.")


✅ 상위 8개 테이블에서 특정 항목 및 모든 금액을 개별 셀로 저장 완료: output_엔클로니_filtered.csv, output_엔클로니_filtered.xlsx


In [16]:
import pdfplumber
import pandas as pd

pdf_path = "/content/우진기전_사보고서.pdf"

data = []
with pdfplumber.open(pdf_path) as pdf:
    for page in pdf.pages:
        tables = page.extract_tables()  # 페이지에서 모든 테이블 추출
        if tables:
            data.extend(tables)  # 리스트 평탄화하여 추가

# 상위 5개 테이블만 선택
data = data[:10]

# 각 테이블을 DataFrame으로 변환 후 하나로 합치기
dfs = [pd.DataFrame(table) for table in data if table]  # 빈 테이블 제외
df_combined = pd.concat(dfs, ignore_index=True)  # 하나의 DataFrame으로 합치기

# 파일 저장
df_combined.to_csv("output_우진기전_combined.csv", index=False, encoding="utf-8-sig")
df_combined.to_excel("output_우진기전_combined.xlsx", index=False)

print("✅ 상위 5개 테이블을 하나의 파일로 저장 완료: output_엔클로니_combined.csv, output_엔클로니_combined.xlsx")


✅ 상위 5개 테이블을 하나의 파일로 저장 완료: output_엔클로니_combined.csv, output_엔클로니_combined.xlsx


In [13]:
import pdfplumber
import pandas as pd

pdf_path = "/content/엔클로니_감사보고서.pdf"

data = []
with pdfplumber.open(pdf_path) as pdf:
    for page_num, page in enumerate(pdf.pages, start=1):
        tables = page.extract_tables()  # 페이지별 테이블 추출
        if tables:
            for table in tables:
                if table:
                    data.append((page_num, table))  # 페이지 번호와 함께 저장

# 📌 상위 5개 테이블 선택 (다른 페이지에 걸쳐 있을 가능성 고려)
selected_tables = data[:8]

# 📌 테이블 합치기 (같은 페이지에서 분할된 경우 연결 가능)
dfs = []
for i, (page_num, table) in enumerate(selected_tables):
    df = pd.DataFrame(table)

    # 첫 번째 행이 컬럼명으로 적절한지 확인 후 설정
    if i == 0:
        df.columns = df.iloc[0]  # 첫 번째 행을 컬럼명으로 설정
        df = df[1:].reset_index(drop=True)  # 첫 번째 행 제거 후 인덱스 재정렬

    dfs.append(df)

# 📌 하나의 DataFrame으로 병합
df_combined = pd.concat(dfs, ignore_index=True)

# 📌 파일 저장
df_combined.to_csv("output_엔클로니_combined.csv", index=False, encoding="utf-8-sig")
df_combined.to_excel("output_엔클로니_combined.xlsx", index=False)

print("✅ 상위 5개 테이블을 하나의 파일로 저장 완료: output_엔클로니_combined.csv, output_엔클로니_combined.xlsx")


InvalidIndexError: Reindexing only valid with uniquely valued Index objects

In [ ]:
import pdfplumber
import pandas as pd
import os

def pdfs_to_excels(pdf_folder, output_folder):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)  # 출력 폴더가 없으면 생성

    # 폴더 내 모든 PDF 파일 읽기
    for file in os.listdir(pdf_folder):
        if file.endswith(".pdf"):  # PDF 파일만 처리
            pdf_path = os.path.join(pdf_folder, file)
            output_excel = os.path.join(output_folder, f"{os.path.splitext(file)[0]}.xlsx")  # PDF와 동일한 이름의 Excel 파일 생성

            # PDF에서 표 추출
            with pdfplumber.open(pdf_path) as pdf:
                all_tables = []
                for page in pdf.pages:
                    tables = page.extract_table()
                    if tables:
                        all_tables.extend(tables)  # 표 데이터를 모두 합침

            if all_tables:  # 표 데이터가 있는 경우
                # 가장 많은 열 수를 파악
                max_columns = max(len(row) for row in all_tables)

                # 각 행에 대해 부족한 열을 None으로 채우기
                for row in all_tables:
                    while len(row) < max_columns:
                        row.append(None)  # 부족한 열을 None으로 채움

                # DataFrame으로 변환
                columns = all_tables[0]  # 첫 번째 행을 컬럼명으로 사용
                df = pd.DataFrame(all_tables[1:], columns=columns)

                df.to_excel(output_excel, index=False)  # 개별 xlsx 파일 저장
                print(f"엑셀 파일 저장 완료: {output_excel}")
            else:
                # 표가 없을 경우 텍스트 추출
                text = "\n".join([page.extract_text() for page in pdf.pages if page.extract_text()])
                df = pd.DataFrame([{"파일명": file, "내용": text}])
                df.to_excel(output_excel, index=False)  # 텍스트만 포함된 엑셀 파일 저장
                print(f"텍스트만 포함된 엑셀 파일 저장 완료: {output_excel}")

# 사용 예시 (Colab 환경에서 실행)
pdfs_to_excels("/content", "/content/excel_outputs")


엑셀 파일 저장 완료: /content/excel_outputs/우진기전_사보고서.xlsx
엑셀 파일 저장 완료: /content/excel_outputs/큐로드_감사보고서.xlsx
엑셀 파일 저장 완료: /content/excel_outputs/엔클로니_감사보고서.xlsx


In [ ]:
import shutil
import os
from google.colab import files

def zip_and_download_folder(folder_path, zip_filename):
    # 폴더 압축
    shutil.make_archive(zip_filename, 'zip', folder_path)

    # 압축된 파일 다운로드
    files.download(f"{zip_filename}.zip")

# 예시: /content/excel_outputs 폴더를 압축하여 다운로드
zip_and_download_folder('/content/excel_outputs', '/content/excel_outputs')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>